In [4]:
import catalog_util_astro as cua
import astropy 
from astropy.table import Table, Column, join
import numpy as np
from astropy.coordinates import SkyCoord
from astropy import units as u

In [2]:
#Typical ETL Process for Catalog Data downloaded from GATOR. 
#The goal is to read glimpse and mipsgal catalog into astropy tables and merge mipsgal table into glimpse table.
#The result should be a table containing objects with magnitude and magnitude uncertainties at 
#3.6, 4.5, 5.8, 8.0 microns, JHK bands and 24 microns 

In [5]:
#1. Read tables from files

#Read glimpse catalog
g = Table.read('/home/kecai/w49/glimpse_demo.tbl', format='ascii')

#Read mipsgal catalog
m = Table.read('/home/kecai/w49/mipsgal_demo.tbl', format='ascii')

In [6]:
#2. Rename columns of the mipsgal table.
#We follow the naming convention of glimpse catalog
m.rename_column('glimpse_name', 'designation')
m.rename_column('mag_24', 'mag24')
m.rename_column('sigma_mag_24', 'd24m')

In [8]:
#3. Strip the 8-character prefix in glimpse designation
for i in range(0, len(g['designation'])):
    g['designation'][i] = g['designation'][i][8:]

In [12]:
#4. Split mipsgal catalog into two parts: 

#a part containing objects that have glimpse names
#this part will be merged into glimpse catalog by designation
ma = m[m['designation'].mask==False]['designation', 'mag24', 'd24m'] 

#a part containing objects that do not have glimpse names
#this part will be merged into glimpse catalog by matching objects with separation less than a user defined threshold.
mb = m[m['designation'].mask==True]['ra', 'dec', 'mag24', 'd24m']

In [14]:
#Use desig_merge function to merge ma into g. 
#desig_merge returns a new table containing merged data
merged = cua.desig_merge(g, ma)

In [15]:
#To merge mb into g, first compute the distance mb and g. 
#Only when it is small enough do we have the need to merge mb into g.  
cua.cat_dist(g, mb)

<Angle 0.0006050000000005055 deg>

In [16]:
#If we do decide to merge mb into g, need to merge mb into the table returned by desig_merge
#The third argument is the threshold separation between objects, in units of spatial_err, such that
#If separation between two objects < threshold * spatial_err, the two objects will be considered the same
#And 24 micron detection in mb will be updated into merged
cua.coord_merge(merged, mb, 1)

#After this step, merged will be the table we can perform analysis on. 

0

In [17]:
g

designation,ra,dec,mag_j,dj_m,mag_h,dh_m,mag_ks,dks_m,mag3_6,d3_6m,mag4_5,d4_5m,mag5_8,d5_8m,mag8_0,d8_0m,color36_45,color36_58,color36_80,color45_58,color45_80,color58_80
,deg,deg,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,,,,
str25,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
G042.3083-00.0663,287.224914,8.306492,11.842,0.023,11.239,0.027,11.024,0.023,10.86,0.057,10.781,0.098,10.755,0.062,10.686,0.07,0.079,0.105,0.174,0.026,0.095,0.069
G042.3109-00.0668,287.226641,8.308583,--,--,14.428,0.077,13.883,0.069,13.552,0.073,13.324,0.136,--,--,--,--,0.228,--,--,--,--,--
G042.3099-00.0668,287.226146,8.307731,--,--,14.62,0.11,14.003,0.106,13.274,0.08,13.192,0.146,--,--,--,--,0.082,--,--,--,--,--
G042.3150-00.0632,287.225257,8.313943,--,--,--,--,--,--,13.798,0.094,13.66,0.167,--,--,--,--,0.138,--,--,--,--,--
G042.3122-00.0662,287.226687,8.309985,--,--,--,--,--,--,12.174,0.046,11.976,0.089,11.745,0.124,--,--,0.198,0.429,--,0.231,--,--
G042.3111-00.0644,287.224573,8.309867,15.909,0.085,15.197,0.103,14.618,0.108,13.764,0.105,--,--,--,--,--,--,--,--,--,--,--,--
G042.3103-00.0627,287.222672,8.30991,--,--,--,--,--,--,13.648,0.084,13.352,0.165,--,--,--,--,0.296,--,--,--,--,--
G042.3300-00.0265,287.199352,8.344129,--,--,--,--,14.369,0.094,13.104,0.071,12.967,0.075,--,--,--,--,0.137,--,--,--,--,--
